## Step 1. Import and connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -U spacy==2.1.0 
!python -m spacy download en
!pip install Cython --install-option="--no-cython-compile"
!pip uninstall -y neuralcoref 
!pip install neuralcoref --no-binary neuralcoref
!git clone https://github.com/thunlp/OpenNRE.git --depth 1
!pip install -r OpenNRE/requirements.txt
!pip install -e OpenNRE/.

     |████████████████████████████████| 27.7MB 131kB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
     |████████████████████████████████| 3.2MB 41.3MB/s 
     |████████████████████████████████| 2.1MB 43.6MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: preshed 3.0.5
    Uninstalling preshed-3.0.5:
      Successfully uninstalled preshed-3.0.5
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 11.1MB 3.0MB/s 
  Created wheel for en-core-web-

In [ ]:
import re
import itertools
import json
import urllib
from string import punctuation
import nltk
import spacy
import neuralcoref
import opennre

## Step 2: Read data from google drive and using regex to remove special character

In [ ]:
path = "/content/gdrive/MyDrive/Colab Notebooks/input/vn_tourism.txt"

In [ ]:
with open(path, 'r') as file:
  filedata = file.read()

In [ ]:
filedata = re.sub('[~@#$%^&*(){}\[\]`\n++©🙂]', ' ', filedata)

## Step 3: Inital all models

In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks/model

/content/gdrive/MyDrive/Colab Notebooks/model


In [ ]:
ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

# Load SpaCy
nlp = spacy.load('en')
# Add neural coref to SpaCy's pipe
neuralcoref.add_to_pipe(nlp)

# Load opennre -- VN Tourism data's quite similar to wiki data
relation_model = opennre.get_model('wiki80_bert_softmax')

# Load NLTK
nltk.download('punkt')

2021-06-10 10:36:16,298 - root - INFO - Loading BERT pre-trained checkpoint.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Step 4: Using neural corefence to improve text

In [ ]:
def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

In [ ]:
corpus = coref_resolution(filedata)

In [ ]:
corpus

'Dive into Hanoi\'s pulsating city streets, and you\'ll capture the essence of Vietnamese life. The country\'s capital is a burgeoning economic center that still clings strongly to traditional culture, managing to be a showcase of both old and modern Vietnam. The old quarter district hums with street vendor action; the cafés and restaurants are vibrant, contemporary scenes; and just trying to cross a road here can end up being an adrenaline-fueled escapade. When the crowds begin to wear you down, Hanoi has a bundle of places to visit where you can escape for some peace. Hoan Kiem Lake is a relaxing respite right within the city, while the Temple of Literature and Vietnam Museum of Ethnology provide plenty of opportunities to reflect on Vietnam\'s grand history. For more ideas on things to do, see our list of the top attractions in Hanoi. See also: Where to Stay in Hanoi Note: Some businesses may be temporarily closed due to recent global health and safety issues.   For many visitors to

## Step 2: Named Entity Linking

In [ ]:
def wikifier(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    results = list()
    for annotation in response["annotations"]:
        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
            elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

## Step 3: Relationship extraction

In [ ]:
def strip_punctuation(s):
    """Removes all punctuation from a string"""
    return ''.join(c for c in s if c not in punctuation)

In [ ]:
def deduplicate_dict(d):
    return [dict(y) for y in set(tuple(x.items()) for x in d)]

In [ ]:
relation_threshold = 0.4
entities_threshold = 0.8
relations_list = list()
entities_list = list()

In [ ]:
for sentence in nltk.sent_tokenize(corpus):
    sentence = strip_punctuation(sentence)
    entities = wikifier(sentence, threshold=entities_threshold)
    entities_list.extend([{'title': el['title'], 'wikiId': el['wikiId'], 'label': el['label']} for el in entities])
    # Iterate over every permutation pair of entities
    for permutation in itertools.permutations(entities, 2):
       for source in permutation[0]['characters']:
           for target in permutation[1]['characters']:
           # Relationship extraction with OpenNRE
              data = relation_model.infer({'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
              if data[1] > relation_threshold:
                  relations_list.append({'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': data[0]})

kg_data =  {'entities': deduplicate_dict(entities_list), 'relations': deduplicate_dict(relations_list)}

In [ ]:
len(kg_data['entities'])

292

In [ ]:
len(kg_data['relations'])

1308

In [ ]:
import pandas as pd

kg = pd.DataFrame(kg_data['relations'])

In [ ]:
kg

source  ...                                              type
0     Russian Aircraft Corporation MiG  ...                                          operator
1                       Atlantic Ocean  ...                        located on terrain feature
2                              Côn Đảo  ...                        located on terrain feature
3        Phu Bai International Airport  ...                                           country
4                      Vietnamese đồng  ...                                          owned by
...                                ...  ...                                               ...
1303                    Atlantic Ocean  ...                                       instance of
1304                           Village  ...  located in the administrative territorial entity
1305                             China  ...                                       followed by
1306                          BBC News  ...                                          has part
1307                            Taoism  ...                                       followed by

[1308 rows x 3 columns]

In [ ]:
kg.to_csv("/content/gdrive/MyDrive/Colab Notebooks/input/TourismVN.csv")